In [1]:
import numpy as np
import scipy
import numpy 
import matplotlib.pyplot as plt
from scipy import linalg, special, stats
from numpy import genfromtxt
import ML_support as ml

In [2]:
class GaussianClassifier:
    def __init__(self):
        self.C = {}
        self.mu = {}

    def train(self, DTR, LTR):
        self.mu, self.C = ml.MU_Cov_calculator(DTR, LTR)

    def test(self, DTE, LTE):
        S = numpy.zeros((numpy.unique(LTE).size, DTE.shape[1]))
        predicted = []

        for i in numpy.unique(LTE):
            S[i, :] = ml.GAU_logpdf_ND(DTE, self.mu[i], self.C[i]) + numpy.log(1 / 2)

        Sp = scipy.special.logsumexp(S, axis=0)

        for x, p in zip(S.T, Sp): 
            tmp = x - p
            predicted.append(numpy.argmax(tmp))

        predicted = numpy.array(predicted)

        True_prediction = numpy.array([predicted == LTE])

        error = 1 - (numpy.count_nonzero(True_prediction) / True_prediction.size)

        print("Gaussian Classifier error:", error)


In [3]:
Data, label = ml.loadFile('../Train.txt')

In [4]:
(DTR, LTR), (DTE,LTE) = ml.split_db_2to1(Data, label, seed=42)

In [5]:
G=GaussianClassifier()
G.train(DTR,LTR)
G.test(DTE,LTE)

Gaussian Classifier error: 0.040644944575075614


In [6]:
Gn=GaussianClassifier()
Gn.train(ml.z_normalization(DTR), LTR)
Gn.test(DTE, LTE)
Gn.test(ml.z_normalization(DTE), LTE)

Gaussian Classifier error: 0.04165267047363119
Gaussian Classifier error: 0.04266039637218677


In [7]:
GG=GaussianClassifier()
GG.train(ml.gaussianize(DTR), LTR)
GG.test(DTE, LTE)
GG.test(ml.gaussianize(DTE), LTE)

Gaussian Classifier error: 0.0923748740342627
Gaussian Classifier error: 0.08296943231441045


In [8]:
class GaussianClassifier_mod1:
    def __init__(self):
        self.C = {}
        self.mu = {}

    def train(self, DTR, LTR):
        self.mu, self.C = ml.MU_Cov_calculator(DTR, LTR)

    def test(self, DTE, LTE, app):
        S = numpy.zeros((numpy.unique(LTE).size, DTE.shape[1]))
        ll = numpy.zeros((numpy.unique(LTE).size, DTE.shape[1]))
        predicted = []

        for i in numpy.unique(LTE):
            ll[i, :] = ml.GAU_logpdf_ND(DTE, self.mu[i], self.C[i])
            S[i, :] = ll[i, :]  + numpy.log(1/2)

        Sp = scipy.special.logsumexp(S, axis=0)

        for x, p in zip(S.T, Sp):
            tmp = x - p
            predicted.append(numpy.argmax(tmp))

        predicted = numpy.array(predicted)

        app_bayes_risk=ml.compute_Bayes_risk(ml.compute_confusion_matrix(predicted, LTE), app)
        DCF = ml.compute_norm_Bayes(app_bayes_risk, app)
        
        llr = numpy.array(ll[1, :]-ll[0, :])
        minDCF= ml.compute_min_DCF(llr, app, LTE)


        True_prediction = numpy.array([predicted == LTE])
        error = 1 - (numpy.count_nonzero(True_prediction) / True_prediction.size)
        
        print("\-/ \-/ \-/ \-/ \-/ ")
        print("Gaussian Classifier error:", error)
        print(app,"DCF:", DCF, "minDCF:", minDCF)
        print("/-\ /-\ /-\ /-\ /-\ ")

In [9]:
G=GaussianClassifier_mod1()
G.train(DTR,LTR)
G.test(DTE,LTE, [1/2,1,1])

\-/ \-/ \-/ \-/ \-/ 
Gaussian Classifier error: 0.040644944575075614
[0.5, 1, 1] DCF: 0.16563353744700895 minDCF: 0.1256510329049189
/-\ /-\ /-\ /-\ /-\ 


In [10]:
G.test(DTE,LTE, [1/3,1,1])

\-/ \-/ \-/ \-/ \-/ 
Gaussian Classifier error: 0.040644944575075614
[0.3333333333333333, 1, 1] DCF: 0.19672162034856336 minDCF: 0.17622501850481126
/-\ /-\ /-\ /-\ /-\ 


In [11]:
G.test(DTE,LTE, [0.1,1,1])

\-/ \-/ \-/ \-/ \-/ 
Gaussian Classifier error: 0.040644944575075614
[0.1, 1, 1] DCF: 0.41433820065944416 minDCF: 0.28024628221519415
/-\ /-\ /-\ /-\ /-\ 


In [ ]:
G.test(DTE,LTE, [0.1,1,1])